In [49]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
import spacy
from tqdm import tqdm
#nlp = spacy.load('en_core_web_lg', parse=True, tag=True, entity=True)
#nlp = spacy.load('en_core_web_sm',parse=True, tag=True, entity=True)
nlp = spacy.load('en')
import nltk
from textblob import TextBlob

In [50]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 

In [51]:
toy_rev = pd.read_csv('csvfile.csv')
toy_rev.head()

toy_rev = toy_rev['Tweets'].to_frame()

C:\Users\sm185567\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [52]:
toy_rev = toy_rev.dropna().reset_index()

In [53]:
toy_rev = toy_rev.drop(['index'],axis=1)

In [54]:
def preprocess(text):
    lines=text
    lines = re.sub('[@#:;?\.]', ' ',lines)
    lines = re.sub('https.*', ' ',lines)
    lines = re.sub('co.*', ' ',lines)
    lines = re.sub('[0-9]', ' ',lines)
    lines = lines.encode('ascii', 'ignore').decode('ascii')
    return lines

In [55]:
hi = toy_rev.Tweets.apply(lambda x: preprocess(x))

In [56]:
hi = hi.to_frame()

In [57]:
polarity_score=[]
subjective_score=[]
for x in tqdm(range(len(hi['Tweets']))):
    analysis = TextBlob(hi['Tweets'][x]) 
    polarity_score.append(analysis.sentiment.polarity)
    subjective_score.append(analysis.sentiment.subjectivity)

100%|██████████████████████████████████████████████████████████████████████████| 78637/78637 [00:11<00:00, 6579.39it/s]


In [58]:
series= pd.Series(polarity_score)
series2 = pd.Series(subjective_score)

In [59]:
df = pd.DataFrame({'tweets':hi['Tweets'],'sentiment_score':polarity_score,'subjective_score':series2})

In [60]:
df

,tweets,sentiment_score,subjective_score
0,RT sanket It may sound somewhat seditious bu...,9.251859e-18,0.266667
1,Mirzapur marathon happening in house,0.000000e+00,0.000000
2,This match was as disappointing as Mirzapur,-6.000000e-01,0.700000
3,Dream IPL MIvRR Mirzapur,0.000000e+00,0.000000
4,When you miss your sales targets! Mirzapur,0.000000e+00,0.000000
...,...,...,...
78632,Le barber --,0.000000e+00,0.000000
78633,Ft Mirzapur,0.000000e+00,0.000000
78634,RT SanDhirParShFC Queen Slaying In Blue Ha...,0.000000e+00,0.100000
78635,HarshitaGaur mirzapurseason Mirzapur,0.000000e+00,0.000000


In [61]:
df.loc[df.sentiment_score>0.0,'sentiment'] = 'Positive'

In [62]:
df.loc[df.sentiment_score==0.0,'sentiment'] = 'Neutral'

In [63]:
df.loc[df.sentiment_score<0.0,'sentiment'] = 'Negative'

In [64]:
df

,tweets,sentiment_score,subjective_score,sentiment
0,RT sanket It may sound somewhat seditious bu...,9.251859e-18,0.266667,Positive
1,Mirzapur marathon happening in house,0.000000e+00,0.000000,Neutral
2,This match was as disappointing as Mirzapur,-6.000000e-01,0.700000,Negative
3,Dream IPL MIvRR Mirzapur,0.000000e+00,0.000000,Neutral
4,When you miss your sales targets! Mirzapur,0.000000e+00,0.000000,Neutral
...,...,...,...,...
78632,Le barber --,0.000000e+00,0.000000,Neutral
78633,Ft Mirzapur,0.000000e+00,0.000000,Neutral
78634,RT SanDhirParShFC Queen Slaying In Blue Ha...,0.000000e+00,0.100000,Neutral
78635,HarshitaGaur mirzapurseason Mirzapur,0.000000e+00,0.000000,Neutral


In [65]:
df.to_csv('textblob.csv')